# Udacity Data Analyst Project 3:  Wrangling Open Street Data

_ by Jens Laufer _

_ jenslaufer@gmail.com _


## Introduction

In this project I am importing XML Data from OpenStreetMap, auditing and wrangling this data and export it to MongoDB. Afterwards I am doing some analysis on the imported data.

I selected the area of Ostfriesland in north eastern Germany, which is close to the north sea. Although I am not from there, I am interested in the area as I want to extend my Airbnb hosting business and the area looks promising to me, as it is touristic and property prices are low. I got these insights about the area from google search data and data from the german realestate website [Immobilienscout24](http://immobilienscout24.de), but this is not part of the assigment. I am especially interesting in all data related to lodging. 

![](img/map.png)

I moved the code from the case study from the Udacity Data Analyst nanodegree course to a module called osm.py, which I am using within the scope of this assignment. I added some additional functions there e.g for auditing the contact data.

In [1]:
# python imports
import pandas as pd
import numpy as np
from collections import defaultdict
import pprint
import re
import codecs
import json
import os
import file_size_humanize as humanize
import os.path as path
import time
from pprint import pprint
import pymongo as mongo
# this is the import of the code from the case study
import osm

In [2]:
"""
definition of constants which I am using in this notebook 
"""

DO_IMPORT = False

OSM_URL = "http://overpass-api.de/api/map?bbox=6.6309,53.4302,7.8291,53.8227"
OSM_FILE = "ostfriesland.osm" 
OSM_EXPORT_FILE = "{0}.json".format(OSM_FILE)

MONGO_URL = 'mongodb://localhost:27017/'

## Download of the dataset

In [3]:
"""
I stream the osm data with the overpass url for the box I want to use into a local file in case the file does not already exist
"""
from urllib2 import urlopen

if not path.exists(OSM_FILE):
    response = urlopen(OSM_URL)
    CHUNK = 16 * 1024
    with open(OSM_FILE, 'wb') as f:
        while True:
            chunk = response.read(CHUNK)
            if not chunk:
                break
            f.write(chunk)

## Auditing of the data

#### Auditing contact data

I am auditing the contact data. Therefore I created a audit_contact_data in the osm module, which checks email address and URLs against regular expressions. For the phonenumber I am using a port of [Google's libphonenumber for python](https://github.com/daviddrysdale/python-phonenumbers) to test for phonenumber validity.

In [4]:
invalid_phone, invalid_email, invalid_url = osm.audit_contact_data(OSM_FILE)

Error parsing 0800 283 50000: (0) Missing or invalid default region.
Error parsing 01622 11 90 66: (0) Missing or invalid default region.
Error parsing 0173 - 292 21 90: (0) Missing or invalid default region.


In [5]:
invalid_phone

['0800 283 50000', '01622 11 90 66', '0173 - 292 21 90']

In [6]:
invalid_email

[]

In [7]:
invalid_url

['www.silvis-bungalow.de',
 'www.haus-thomas.de',
 'www.tuedelpott.de',
 'www.hotel-cafecaro.de',
 'www.hotel-westfalenhof.de']

I need to fix this problematic data, before exporting the data to MongoDB.

## Export of OSM to JSON and Import to MongoDB

I am extracting the data I am interested to, fixing the problematic data entries and export it to JSON for the MongoDB import.

In [8]:
start = time.time()

if DO_IMPORT:
    osm.process(OSM_FILE,OSM_EXPORT_FILE);
    
time.time() - start

0.0

I am adding an index on id to improve speed on upsert on the id field

In [9]:
nodes = mongo.MongoClient(MONGO_URL).osm.nodes
nodes.create_index([("id", mongo.ASCENDING)]);


In [10]:
start = time.time()

if DO_IMPORT:
    os.system('mongoimport --quiet --upsertFields id --db osm --collection nodes --file {0}'.format(OSM_FILE+'.json'));
    
time.time() - start

0.0

### Addition of some indexes

In [88]:
nodes.create_index([("type", mongo.ASCENDING)])
nodes.create_index([("address.city", mongo.ASCENDING)])

nodes.create_index([("pos", mongo.GEOSPHERE)]);

## Analysis of the data

#### Tags in OSM file

In [12]:
pprint(osm.count_tags(OSM_FILE))

{'bounds': 1,
 'member': 48150,
 'meta': 1,
 'nd': 878995,
 'node': 724983,
 'note': 1,
 'osm': 1,
 'relation': 877,
 'tag': 346155,
 'way': 103280}


#### File sizes

In [13]:
# File size of the full osm file
info = os.stat(OSM_FILE)
"Filesize of {0} {1}".format(OSM_FILE, humanize.humansize(info.st_size))

'Filesize of ostfriesland.osm 155.29 MB'

In [14]:
# File size of the full osm file
info = os.stat(OSM_EXPORT_FILE)
"Filesize of {0} {1}".format(OSM_EXPORT_FILE, humanize.humansize(info.st_size))

'Filesize of ostfriesland.osm.json 156.26 MB'

#### Number of documents in database

In [15]:
"{} Documents in MongoDB".format(nodes.find().count())

'828259 Documents in MongoDB'

#### Example document in MongoDB

In [16]:
pprint(nodes.find_one({ "contact.phone": { '$exists': 1 }, "contact.fax": { '$exists': 1 }, 
            "contact.website": { '$exists': 1 }, "address.street": { '$exists': 1 }, 
            "address.city": { '$exists': 1 } }))

{u'_id': ObjectId('58ac61eedd946dd35d3761c6'),
 u'address': {u'city': u'Aurich',
              u'country': u'DE',
              u'housenumber': u'40',
              u'postcode': u'26607',
              u'street': u'Dornumer Stra\xdfe'},
 u'contact': {u'email': u'info@antik-alteschmiede.de',
              u'fax': u'+49 4941 71025',
              u'phone': u'+49 4941 7915',
              u'website': u'http://www.antik-alteschmiede.de'},
 u'created': {u'changeset': u'39414176',
              u'timestamp': u'2016-05-19T02:00:25Z',
              u'uid': u'52533',
              u'user': u'imehl',
              u'version': u'1'},
 u'id': u'4194453896',
 u'name': u'Antik "Alte Schmiede"',
 u'pos': [53.5078165, 7.4807148],
 u'type': u'node'}


#### Document types

In [17]:
pd.DataFrame(list(nodes.aggregate([
        {'$group':{'_id':'$type','count':{'$sum':1}}}
    ])))

,_id,count
0,way,103280
1,node,724979


#### Documents with contact data

In [18]:
"{} Documents with contact in MongoDB".format(nodes.find({'contact':{'$exists':1}}).count())

'267 Documents with contact in MongoDB'

#### Documents with address data

In [19]:
"{} Documents with address in MongoDB".format(nodes.find({'address':{'$exists':1}}).count())

'22542 Documents with address in MongoDB'

#### Test Fixing of problematic data

I am checking if my code for importing data fixed the problematic phone numbers and URLs.

In [20]:
nodes.find({'contact.phone':{'$in':invalid_phone}}).count()

0

In [21]:
nodes.find({'contact.phone':{'$in':['+49 800 28350000', '+49 162 2119066', '+49 173 2922190']}}).count()

3

In [22]:
nodes.find({'contact.website':{'$in':invalid_url}}).count()

0

In [23]:
nodes.find({'contact.website':{'$in':
['http://www.silvis-bungalow.de',
 'http://www.haus-thomas.de',
 'http://www.tuedelpott.de',
 'http://www.hotel-cafecaro.de',
 'http://www.hotel-westfalenhof.de']}}).count()

5

All roblematic contact data was fixed correctly.

#### Analysis of lodging data

What type of touristic data is there?

In [24]:
pd.DataFrame(list(nodes.aggregate([
   {'$match': {'tourism':{'$exists':1}}},
   {'$group':{'_id':'$tourism'}}
        ])))

,_id
0,theme_park
1,artwork
2,caravan_site
3,zoo
4,trail_riding_station
5,museum
6,motel
7,gallery
8,apartment
9,hostel


I am interested in the number of lodging facilities in the area:

In [78]:
pd.DataFrame(list(nodes.aggregate([
                {'$match':{'tourism':{'$in':['bed_and_breakfast','motel','apartment','hostel','guest_house','chalet','hotel']}}},
                {'$group':{'_id':'$address.city', 'count':{'$sum':1}}},
                {'$sort':{'count':-1}}
            ])))

,_id,count
0,None,127
1,Norden,112
2,Krummhörn,50
3,Baltrum,44
4,Juist,37
5,Norderney,32
6,Borkum,31
7,Langeoog,20
8,Spiekeroog,20
9,Neuharlingersiel,18


It is interesting, that for 127 lodging facilities no city is in the data set.

As I have the opportunity to buy a property in the village of Dornum I a doing a little data analysis with other data I scraped from [Airbnb](http://airbnb.com) and from  the german realestate website [Immobilienscout24](http://immobilienscout24.de) with their API:

#### Comparing airbnb listing data with data from OSM

I am extracting all airbnb listing for about the same area:


In [101]:

pd.DataFrame(list(nodes.aggregate(
            [
                {
                    '$geoNear':
                    {
                        'near':
                        {
                            'type': 'Point',
                            'coordinates':[53.645903, 7.430451]
                        },
                        'spherical': True,
                        'query': {
                            'tourism':{'$in':['bed_and_breakfast','motel','apartment','hostel','guest_house','chalet','hotel']}},
                        'maxDistance' : 5000,
                        'distanceField':'dist',
                    },
                }
            ])))

,_id,amenity,created,dist,id,name,pos,tourism,type
0,58ac61c7dd946dd35d32b823,NaN,"{u'changeset': u'22128404', u'version': u'1', ...",196.249016,2836156560,Beningaburg,"[53.6475113, 7.4296996]",hotel,node
1,58ac617fdd946dd35d2ac6b8,restaurant,"{u'changeset': u'11413650', u'version': u'2', ...",201.714203,1729255586,Dornumer Wappen,"[53.6474635, 7.4313939]",hotel,node


In [104]:

listings = mongo.MongoClient(MONGO_URL).airbnb.listings

pd.DataFrame(list(listings.aggregate([ 
                {
                    '$geoNear':
                    {
                        'near':
                        {
                            'type': 'Point',
                            'coordinates':[53.645903, 7.430451]
                        },
                        'spherical': True,
                        'query': {},
                        'maxDistance' : 5000,
                        'distanceField':'dist',
                    },
                }
    ])))

,_id,bathrooms,bedrooms,beds,city,dist,distance,extra_host_languages,geography,id,...,room_type,room_type_category,scrim_color,star_rating,thumbnail_url,ufi,user,user_id,xl_picture_url,xl_picture_urls
0,58ac16badd946dd35d147be9,1.0,3,3,Dornum,159.949776,None,[],"{u'city': u'Dornum', u'loc': [53.6459028, 7.43...",9418724,...,Ganze Unterkunft,entire_home,#86101E,NaN,https://a0.muscache.com/im/pictures/f1abc8ec-e...,-1761007,"{u'first_name': u'Melanie', u'picture_url': u'...",24692863,https://a0.muscache.com/im/pictures/f1abc8ec-e...,[https://a0.muscache.com/im/pictures/f1abc8ec-...
1,58ac16badd946dd35d147c0f,2.0,3,4,Dornum,252.444474,None,[],"{u'city': u'Dornum', u'loc': [53.6459028, 7.43...",5408490,...,Ganze Unterkunft,entire_home,#45351C,NaN,https://a0.muscache.com/im/pictures/67515622/6...,-1761007,"{u'first_name': u'Melanie', u'picture_url': u'...",24692863,https://a0.muscache.com/im/pictures/67515622/6...,[https://a0.muscache.com/im/pictures/67515622/...
2,58ac16badd946dd35d147c13,1.0,2,3,Dornum,1403.769869,None,[],"{u'city': u'Dornum', u'loc': [53.6459028, 7.43...",5450067,...,Ganze Unterkunft,entire_home,#7B4731,NaN,https://a0.muscache.com/im/pictures/67988981/d...,-1761007,"{u'first_name': u'Melanie', u'picture_url': u'...",24692863,https://a0.muscache.com/im/pictures/67988981/d...,[https://a0.muscache.com/im/pictures/67988981/...
3,58ac16badd946dd35d147bcf,1.0,1,3,Dornum,1603.017747,None,[],"{u'city': u'Dornum', u'loc': [53.6459028, 7.43...",14080554,...,Privatzimmer,private_room,#2C3119,5.0,https://a0.muscache.com/im/pictures/465e3c8e-e...,-1761007,"{u'first_name': u'Elisabeth', u'picture_url': ...",84478892,https://a0.muscache.com/im/pictures/465e3c8e-e...,[https://a0.muscache.com/im/pictures/465e3c8e-...
4,58ac16badd946dd35d147bbd,1.0,1,3,Dornum,1643.348852,None,[],"{u'city': u'Dornum', u'loc': [53.6459028, 7.43...",2664718,...,Ganze Unterkunft,entire_home,#171518,4.5,https://a0.muscache.com/im/pictures/61513296/5...,-1761007,"{u'first_name': u'Hildegard', u'picture_url': ...",13638564,https://a0.muscache.com/im/pictures/61513296/5...,[https://a0.muscache.com/im/pictures/61513296/...
5,58ac16badd946dd35d147bcd,1.0,2,4,Dornum,1823.142982,None,[],"{u'city': u'Dornum', u'loc': [53.6459028, 7.43...",15387901,...,Ganze Unterkunft,entire_home,#302B27,NaN,https://a0.muscache.com/im/pictures/bc33586e-d...,-1761007,"{u'first_name': u'Klaus', u'picture_url': u'ht...",98376675,https://a0.muscache.com/im/pictures/bc33586e-d...,[https://a0.muscache.com/im/pictures/bc33586e-...
6,58ac16badd946dd35d147c03,1.0,2,3,Dornum,3042.095285,None,[],"{u'city': u'Dornum', u'loc': [53.6459028, 7.43...",5313473,...,Ganze Unterkunft,entire_home,#54534E,NaN,https://a0.muscache.com/im/pictures/66380087/b...,-1761007,"{u'first_name': u'Melanie', u'picture_url': u'...",24692863,https://a0.muscache.com/im/pictures/66380087/b...,[https://a0.muscache.com/im/pictures/66380087/...
7,58ac16badd946dd35d147be5,2.0,2,3,Dornum,3089.866318,None,[],"{u'city': u'Dornum', u'loc': [53.6459028, 7.43...",5342860,...,Ganze Unterkunft,entire_home,#222B26,NaN,https://a0.muscache.com/im/pictures/66737623/4...,-1761007,"{u'first_name': u'Melanie', u'picture_url': u'...",24692863,https://a0.muscache.com/im/pictures/66737623/4...,[https://a0.muscache.com/im/pictures/66737623/...
8,58ac16badd946dd35d147bf5,2.0,2,3,Dornum,3098.137343,None,[],"{u'city': u'Dornum', u'loc': [53.6459028, 7.43...",5460815,...,Ganze Unterkunft,entire_home,#40483B,NaN,https://a0.muscache.com/im/pictures/68118473/a...,-1761007,"{u'first_name': u'Melanie', u'picture_url': u'...",24692863,https://a0.muscache.com/im/pictures/68118473/a...,[https://a0.muscache.com/im/pictures/68118473/...


## Additional problems in the data

  - Cross field consistecy Postcode ork for Street and City. 
  - Have address always a housenumber in case ther is a street
  - Handlingof P.O. boxes
  - E-Mail address still valid
  - Website still available
  - Street names
  - City names
  - Phone number: In case we don't have the country how to get international format?